In [1]:
import ollama

In [13]:
subjects = {
    "Science":{
     "Teacher":"Rajesh Singh",
     "Marks":100,
     "hours_allocated":40
    },
    "Math":{
     "Teacher":"Sumit Sharma",
     "Marks":100,
     "hours_allocated":45
    },
    "English":{
     "Teacher":"Sonal Gupta",
     "Marks":50,
     "hours_allocated":25
    }
}

In [14]:
def get_details(main : str , sub : str) -> dict:
    return {
        main:main,
        sub : subjects.get(main).get(sum)
    }



In [ ]:
system_prompt = {

"role":"system",
"content":    
"""
You are an helpful assitant. when user ask question for details regarding subject and if needed you can call a funcion 'get_details'.

Following is the stucture of the dictionary:

 "English":{
     "Teacher":"Sonal Gupta",
     "Marks":50,
     "hours_allocated":25
    }

The database has data about other subject as well with same stucture.

Suppose a user ask:

Tell me who teaches english ?

then you take English and Teacher as argument.

similarly when a user ask

how many hours has been allocated for math

then you take Math and hours_allocated as argument.

"""
}


In [ ]:
user_prompt = {
    "role":"user",
"content":"how many hours has been allocated to english ?"
}

In [26]:

messages = [system_prompt,user_prompt]

In [27]:
messages

[{'role': 'system',
  'content': '\nYou are an helpful assitant. when user ask question for details regarding subject and if needed you can call a funcion \'get_details\'.\n\nFollowing is the stucture of the dictionary:\n\n "English":{\n     "Teacher":"Sonal Gupta",\n     "Marks":50,\n     "hours_allocated":25\n    }\n\nThe database has data about other subject as well with same stucture.\n\nSuppose a user ask:\n\nTell me who teaches english ?\n\nthen you take English and Teacher as argument.\n\nsimilarly when a user ask\n\nhow many hours has been allocated for math\n\nthen you take Math and hours_allocated as argument.\n\n'},
 {'role': 'user',
  'content': 'how  many hours has been allocated to english ?'}]

In [28]:
response = ollama.chat(
    model='llama3.1', 
    messages=messages,
    tools=[get_details]  # pass the actual function object as a tool
)

In [29]:
response

ChatResponse(model='llama3.1', created_at='2025-12-14T07:58:06.4981714Z', done=True, done_reason='stop', total_duration=8485093500, load_duration=344848900, prompt_eval_count=285, prompt_eval_duration=3261887600, eval_count=24, eval_duration=4741431100, message=Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=[ToolCall(function=Function(name='get_details', arguments={'main': 'English', 'sub': 'hours_allocated'}))]), logprobs=None)

In [30]:
""" (Continuing from previous code) """
available_functions = {"get_details":get_details}

""" Model's initial response after possibly invoking the tool """
assistant_reply = response.message.content
print("Assistant (initial):", assistant_reply)

""" If a tool was called, handle it """
for tool_call in (response.message.tool_calls or []):
    func = available_functions.get(tool_call.function.name)
    if func:
        result = func(**tool_call.function.arguments)
        # Provide the result back to the model in a follow-up message
        messages.append({"role": "assistant", "content": f"The result is {result}."})
        messages.append({"role":"user","content":"what is answer ?"})
        follow_up = ollama.chat(model='llama3.1', messages=messages)
        print("Assistant (final):", follow_up.message.content)

Assistant (initial): 
Assistant (final): To get the answer, I'll call the function `get_details()` with "English" and "hours_allocated" as arguments.

Here's the result:
```
25
```
